# Решение теории

In [ ]:
# covid_data — данные о показателях заболеваемости за каждый день в различных регионах 187 стран (источник данных)

In [2]:
import pandas as pd
covid_data = pd.read_csv('data/covid_data.csv')
display(covid_data.head())

,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


In [ ]:
# Мы будем работать со следующими столбцами:

#date — дата наблюдения;
#province/state — наименование провинции/штата;
#country — наименование страны;
#confirmed — общее число зафиксированных случаев на указанный день;
#deaths — общее число зафиксированных смертей на указанный день;
#recovered — общее число выздоровлений на указанный день.

In [ ]:
# country_vaccinations — данные о процессе вакцинирования людей в 221 стране мира (источник данных)

# Обратите внимание, что в приведённом источнике данные о вакцинации постоянно обновляются. В данном модуле будет использоваться файл, который мы скачали заранее.

In [8]:
import pandas as pd
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
print(vaccinations_data.columns) 

Index(['country', 'iso_code', 'date', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated',
       'daily_vaccinations_raw', 'daily_vaccinations',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'daily_vaccinations_per_million',
       'vaccines', 'source_name', 'source_website'],
      dtype='object')


In [11]:
vaccinations_data = vaccinations_data[
        ['country', 'date', 'total_vaccinations', 
         'people_vaccinated', 'people_vaccinated_per_hundred',
         'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
         'daily_vaccinations', 'vaccines']
    ]
print(vaccinations_data)

           country        date  total_vaccinations  people_vaccinated  \
0      Afghanistan  2021-02-22                 0.0                0.0   
1      Afghanistan  2021-02-23                 NaN                NaN   
2      Afghanistan  2021-02-24                 NaN                NaN   
3      Afghanistan  2021-02-25                 NaN                NaN   
4      Afghanistan  2021-02-26                 NaN                NaN   
...            ...         ...                 ...                ...   
42790     Zimbabwe  2021-09-01           4270430.0          2615233.0   
42791     Zimbabwe  2021-09-02           4323735.0          2649505.0   
42792     Zimbabwe  2021-09-03           4372216.0          2681657.0   
42793     Zimbabwe  2021-09-04           4400246.0          2698332.0   
42794     Zimbabwe  2021-09-05           4431599.0          2713248.0   

       people_vaccinated_per_hundred  people_fully_vaccinated  \
0                               0.00                      

In [ ]:
# Данная таблица содержит следующие столбцы:

country — наименование страны;
date — дата наблюдения;
total_vaccinations — общее число введённых вакцин в стране на указанный день;
people_vaccinated — общее число привитых первым компонентом в стране на указанный день;
people_vaccinated_per_hundred — процент привитых первым компонентом в стране на указанный день (рассчитывается как );
people_fully_vaccinated — общее число привитых вторым компонентом в стране на указанный день (первый компонент уже был введён им ранее);
people_fully_vaccinated_per_hundred — процент привитых вторым компонентом в стране на указанный день (рассчитывается как );
daily_vaccination — ежедневная вакцинация (число вакцинированных в указанный день);
vaccines — комбинации вакцин, используемые в стране.

In [ ]:
# Немного об особенностях данных

# В таблице covid_data данные о заболеваемости представлены для каждого региона (штата) страны за каждый день. Например, для США (United States) записи за 29 мая 2021 года имеют вид:

In [ ]:
# В таблице vaccinations_data параметры вакцинирования отображены для всех стран на каждый день, без учёта регионов. 
# Вот часть таблицы для тех же Соединённых Штатов за 29 мая 2021 года. Как видите, она всего одна:

In [ ]:
# В данных много пропусков, а также неверно введённых значений. Например, для США (United States) и Великобритании (United Kingdom) 
# в большинстве записей таблицы covid_data число выздоровевших людей равно 0. В данном модуле мы закроем на это глаза, но вообще эти страны подлежали бы исключению из данных, 
# или нам понадобился бы дополнительный источник данных, из которого мы смогли бы получить недостающую информацию.
# Обратите внимание, что признаки confirmed, deaths, recovered, total_vaccination, people_vaccinated, people_fully_vaccinated — это суммарные показатели по стране, 
# то есть с каждым днём они должны расти. Такие признаки называют накопительными.
# Возникает большое желание объединить таблицы. Для этого необходимо учитывать следующие нюансы:
# В таблице covid_data необходимо предварительно рассчитать суммарное ежедневное число заболевших во всех провинциях/штатах в каждой стране.
# В таблицах не совпадает число стран, а иногда и их названия. 
# При объединении таблиц по столбцу мы определённо теряем данные (в данной задаче потери незначительны). Избежать этого можно ручными преобразованиями данных — 
# искать различия в названиях стран в таблицах и преобразовывать их. Однако это не является темой данного модуля.
# Таблицы имеют разные периоды наблюдений (вакцины появились позже, чем сам вирус). Объединив данные с типом inner, мы можем потерять большое количество наблюдений в таблице covid_data.

In [13]:
# Предобработка данных
# Опираясь на замечания выше, выполним небольшую предобработку.
# В таблице covid_data:
# Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам. Тем самым переходим от данных по регионам к данным по странам:
covid_data = covid_data.groupby(
        ['date', 'country'], 
        as_index=False
    )[['confirmed', 'deaths', 'recovered']].sum()
print(covid_data)

             date             country  confirmed  deaths  recovered
0      01/01/2021         Afghanistan    51526.0  2191.0    41727.0
1      01/01/2021             Albania    58316.0  1181.0    33634.0
2      01/01/2021             Algeria    99897.0  2762.0    67395.0
3      01/01/2021             Andorra     8117.0    84.0     7463.0
4      01/01/2021              Angola    17568.0   405.0    11146.0
...           ...                 ...        ...     ...        ...
86780  12/31/2020             Vietnam     1465.0    35.0     1325.0
86781  12/31/2020  West Bank and Gaza   138004.0  1400.0   117183.0
86782  12/31/2020               Yemen     2099.0   610.0     1394.0
86783  12/31/2020              Zambia    20725.0   388.0    18660.0
86784  12/31/2020            Zimbabwe    13867.0   363.0    11250.0

[86785 rows x 5 columns]


In [15]:
# Преобразуем даты в формат datetime с помощью функции pd.to_datetime():

covid_data['date'] = pd.to_datetime(covid_data['date'])
print(covid_data['date'])

0       2021-01-01
1       2021-01-01
2       2021-01-01
3       2021-01-01
4       2021-01-01
           ...    
86780   2020-12-31
86781   2020-12-31
86782   2020-12-31
86783   2020-12-31
86784   2020-12-31
Name: date, Length: 86785, dtype: datetime64[ns]


In [17]:
# Создадим признак больных на данный момент (active). Для этого вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов:

covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']
print(covid_data['active'])

0         7608.0
1        23501.0
2        29740.0
3          570.0
4         6017.0
          ...   
86780      105.0
86781    19421.0
86782       95.0
86783     1677.0
86784     2254.0
Name: active, Length: 86785, dtype: float64


In [18]:
# Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. 
# Для этого отсортируем данные по названиям стран, а затем по датам. После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода diff():
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

In [19]:
# Итоговый вид таблицы covid_data:
print(covid_data)

            date                         country  confirmed  deaths  \
11337 2020-02-24                     Afghanistan        1.0     0.0   
11570 2020-02-25                     Afghanistan        1.0     0.0   
11807 2020-02-26                     Afghanistan        1.0     0.0   
12051 2020-02-27                     Afghanistan        1.0     0.0   
12299 2020-02-28                     Afghanistan        1.0     0.0   
...          ...                             ...        ...     ...   
15933 2020-03-12  occupied Palestinian territory        0.0     0.0   
16595 2020-03-14  occupied Palestinian territory        0.0     0.0   
16938 2020-03-15  occupied Palestinian territory        0.0     0.0   
17290 2020-03-16  occupied Palestinian territory        0.0     0.0   
17645 2020-03-17  occupied Palestinian territory        0.0     0.0   

       recovered  active  daily_confirmed  daily_deaths  daily_recovered  
11337        0.0     1.0              NaN           NaN              NaN

In [21]:
# В таблице vaccinations_data достаточно будет преобразовать столбцы в формат datetime:

vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])
print(vaccinations_data['date'])

0       2021-02-22
1       2021-02-23
2       2021-02-24
3       2021-02-25
4       2021-02-26
           ...    
42790   2021-09-01
42791   2021-09-02
42792   2021-09-03
42793   2021-09-04
42794   2021-09-05
Name: date, Length: 42795, dtype: datetime64[ns]


In [ ]:
# Задание 3.1
# За какой период представлены данные в таблице covid_data? В качестве ответа введите даты в формате datetime (без указания времени).
# Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД

In [22]:
print(f"Данные о заболеваемости предоставлены от {covid_data['date'].min()} до  {covid_data['date'].max()}")

Данные о заболеваемости предоставлены от 2020-01-22 00:00:00 до  2021-05-29 00:00:00


In [23]:
# Задание 3.2
# За какой период представлены данные в таблице vaccinations_data? В качестве ответа введите даты в формате datetime без указания времени.
# Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД

In [24]:
print(f"Данные о вакцинации предоставлены от {vaccinations_data['date'].min()} до  {vaccinations_data['date'].max()}")

Данные о вакцинации предоставлены от 2020-12-02 00:00:00 до  2021-09-06 00:00:00


In [ ]:
# Задание 3.3
# С помощью метода merge() объедините таблицы covid_data и vaccinations_data по столбцам date и country.
# Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, вычисленный в задании 3.1. То есть в результирующую таблицу должны попасть все записи из таблицы covid_data и из её пересечения с vaccinations_data, но не более. Результат объединения занесите в переменную covid_df.
# Сохраните таблицу covid_df — она понадобится нам в следующем юните.
# Сколько строк и столбцов в таблице covid_df?
# Введите ответ в виде двух чисел через дефис (например, 333-33): первое число — количество строк, второе число — количество столбцов.

In [25]:
covid_df = covid_data.merge(vaccinations_data, on=['date', 'country'], how='left')
print('Число строк: ', covid_df.shape[0])
print('Число столбцов: ', covid_df.shape[1])

Число строк:  86785
Число столбцов:  16


In [ ]:
# В получившейся в задании 3.3 таблице covid_df создайте признаки death_rate — общий процент смертей среди зафиксированных случаев (летальность) 
# и recover_rate — общий процент случаев выздоровления. 
# Данные характеристики рассчитайте как отношение числа смертей (deaths) и числа выздоровлений (recovered) к числу зафиксированных случаев (confirmed) и умножьте результаты на 100%.

In [ ]:
# Задание 3.4
# Какова максимальная летальность в США (United States) за весь период? Ответ округлите до второго знака после запятой.

In [26]:
covid_df['death_rate'] = covid_df['deaths'] / covid_df['confirmed'] * 100
covid_df['recover_rate'] = covid_df['recovered'] / covid_df['confirmed'] * 100
round(covid_df[covid_df['country'] == 'United States']['death_rate'].max(), 2)

7.19

In [ ]:
# Задание 3.5
# Чему равен средний процент выздоровевших в России (Russia)? Ответ округлите до второго знака после запятой.

In [27]:
round(covid_df[covid_df['country'] == 'Russia']['recover_rate'].mean(), 2)

67.06